In [14]:
import numpy as np
import os

TS = np.load(os.path.join('dump', 'ts.npy'))
DT = np.load(os.path.join('dump', 'dt.npy'))
BT = np.load(os.path.join('dump', 'bt.npy'))
LM = np.load(os.path.join('dump', 'lm.npy'))
MS = np.load(os.path.join('dump', 'ms.npy'))
ID = np.load(os.path.join('dump', 'id.npy'))

In [15]:
print(
    TS.shape, TS.dtype, '\n',
    DT.shape, DT.dtype, '\n',
    ID.shape, ID.dtype, '\n',
    BT.shape, BT.dtype, '\n',
    LM.shape, LM.dtype, '\n',
    MS.shape, MS.dtype, '\n',
)

(10281921,) float64 
 (10281921, 7) int64 
 (10281921,) int64 
 (10281921, 47) float32 
 (10281921, 5) float32 
 (10281921, 4) int64 



In [3]:
import dill
with open('Dolgoprudnyj_gridded.dump', 'rb') as dump:
    meteosonde_data = dill.load(dump)

In [4]:
import sys
sys.path.append('../atmrad')

In [5]:
from cpu.atmosphere import Atmosphere, avg
from cpu.weight_funcs import krho
from cpu.core.static.weight_funcs import kw

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
Q_real = {}
keys = list(meteosonde_data.keys())
for i, key in enumerate(keys):
    T, P, rho_rel, alt = meteosonde_data[key]
    Atm_real = Atmosphere(T, P, RelativeHumidity=rho_rel, altitudes=alt)
    Q_real[key] = Atm_real.Q  # real TWV
    print('\r{:.5f}%'.format((i+1) / len(keys) * 100), end='      ', flush=True)
print('\n')

75.03675%                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



100.00000%                                                                                                                                                                                                                                                                                                                                                     



In [7]:
QREAL = []
for i, _ in enumerate(TS):
    try:
        QREAL.append(Q_real[tuple(MS[i])])
    except KeyError:
        QREAL.append(-1)
    if not i % 10000:
        print('\r{:.5f}%'.format((i+1) / len(TS) * 100), end='      ', flush=True)
QREAL = np.asarray(QREAL, dtype=np.float32)
QREAL

99.99202%                                                                                                                                                                                                                                                                                                                 

array([0.8485806 , 0.8485806 , 0.8485806 , ..., 0.92137563, 0.92137563,
       0.92137563], dtype=float32)

In [8]:
cond = QREAL > 0
QREAL[cond].shape

(10281921,)

In [9]:
TS, DT, ID, BT, LM, MS = TS[cond], DT[cond], ID[cond], BT[cond], LM[cond], MS[cond]

In [10]:
def dump(_obj, _name, _dump_options):
    if not os.path.exists('dump'):
        os.makedirs('dump')
    print('{}...'.format(_name))
    if 'numpy' in _dump_options:
        np.save(os.path.join('dump', '{}.npy'.format(_name)), _obj)
        print('...numpy')
    if 'dill' in _dump_options:
        with open(os.path.join('dump', '{}.dump'.format(_name)), 'wb') as _dump:
            dill.dump(_obj, _dump, recurse=True)
        print('...dill')

In [11]:
dump(_obj=np.asarray(TS), _name='ts', _dump_options=['numpy'])
dump(_obj=np.asarray(DT, dtype=int), _name='dt', _dump_options=['numpy'])
dump(_obj=np.asarray(BT, dtype=np.float32), _name='bt', _dump_options=['numpy'])
dump(_obj=np.asarray(LM, dtype=np.float32), _name='lm', _dump_options=['numpy'])
dump(_obj=np.asarray(MS, dtype=int), _name='ms', _dump_options=['numpy'])
dump(_obj=np.asarray(ID, dtype=int), _name='id', _dump_options=['numpy'])

dump(_obj=np.asarray(TS), _name='ts', _dump_options=['dill'])
dump(_obj=np.asarray(DT, dtype=int), _name='dt', _dump_options=['dill'])
dump(_obj=np.asarray(BT, dtype=np.float32), _name='bt', _dump_options=['dill'])
dump(_obj=np.asarray(LM, dtype=np.float32), _name='lm', _dump_options=['dill'])
dump(_obj=np.asarray(MS, dtype=int), _name='ms', _dump_options=['dill'])
dump(_obj=np.asarray(ID, dtype=int), _name='id', _dump_options=['dill'])

ts...
...numpy
dt...
...numpy
bt...
...numpy
lm...
...numpy
ms...
...numpy
id...
...numpy
ts...
...dill
dt...
...dill
bt...
...dill
lm...
...dill
ms...
...dill
id...
...dill


In [12]:
QREAL = QREAL[cond]

In [13]:
dump(_obj=np.asarray(QREAL), _name='qreal', _dump_options=['numpy', 'dill'])

qreal...
...numpy
...dill
